# Исследование надёжности заёмщиков
**Заказчик — кредитный отдел банка.** Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Данные, с которыми предстоит работать:**

    -children — количество детей в семье
    -days_employed — общий трудовой стаж в днях
    -dob_years — возраст клиента в годах
    -education — уровень образования клиента
    -education_id — идентификатор уровня образования
    -family_status — семейное положение
    -family_status_id — идентификатор семейного положения
    -gender — пол клиента
    -income_type — тип занятости
    -debt — имел ли задолженность по возврату кредитов
    -total_income — ежемесячный доход
    -purpose — цель получения кредита
    
**План выполнения проекта:**

    Шаг 1. Загрузка данных и учение общей информации¶
    Шаг 2. Предобработка данных
        - Проверить типы данных;
        - Поработать с дубликатами;
        - Выделить леммы;
        - Категоризировтаь данные;
        - Выявить зависимости возврата кредита вовремя от признаков;
    Шаг 3. Ответы на вопросы:
        - Есть ли зависимость между наличием детей и возвратом кредита в срок?
        - Есть ли зависимость между семейным положением и возвратом кредита в срок?
        - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
        - Как разные цели кредита влияют на его возврат в срок?    
    Шаг 4. Общий вывод.

## Шаг 1. Загрузка данных и учение общей информации

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()
data.head(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод** 
<br>Файл содержит 12 колонок с информацией о заемщиках. Из них 2 столбца типа float64, 5 столбцов типа int64 и 5 столбцов типа object. В датафрейфме имеются пропуски значений в days_employed, total_income, исследуем их на следующем этапе.


## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
#методом .isna() находим пропуски и .sum() суммируем их
data.isna().sum() 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [3]:
#заменяем пропущенные значения на средние значения в 'total_income'
data_income = data.groupby('income_type')['total_income'].transform('mean')  
data['total_income'] = data['total_income'].fillna(data_income)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
#заменяем пропущенные значения на средние значения в 'days_employed'
data_employed = data.groupby('income_type')['days_employed'].transform('mean')  
data['days_employed'] = data['days_employed'].fillna(data_employed).abs()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод** Без обработки пропусков, теряются важные значения, поэтому б

### Замена типа данных

In [5]:
#методом.astype() переводим данные в нужный нам тип
data['days_employed'] = data['days_employed'].astype('int') 
data['total_income'] = data['total_income'].astype('int')
data.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод** Незамененые тип данных приводит к потере данных


### Обработка дубликатов

In [6]:
#методом .str.lower() в текстовом столбце 'education' и 'family_status' приведу к нижнему регистру
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data.drop_duplicates().reset_index(drop=True)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21452,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


In [7]:
#методом .duplicated() находим кол-во дубликатов
data.duplicated().sum() 

71

In [8]:
#методом.drop_duplicates() избавляемся от них
data = data.drop_duplicates() 
data.duplicated().sum()

0

**Вывод** 
В столбце "education" много одинаковых по смыслу значений, но в разном регистре. Это возможно из-за того, что поле "Образование" при заполнении пользователяим не валидировалось и не приводило все заполненные данные в однообразный вид.
Переведем все строковые значения в один регистр. Дубликаты во всем датафрейме найдены с помощью метода duplicated() в сочетании с методом sum(). Он вернул 71 дубликат. Для удаления использовался метод drop_duplicates().

### Лемматизация

In [9]:
from pymystem3 import Mystem
m = Mystem()
lemmas = data['purpose'].apply(m.lemmatize)
lemmas
word = [] # Цикл для подсчета лемм
for index in lemmas:
    word.extend(index)
from collections import Counter
count_lem = Counter(word)
display(count_lem)

Counter({'покупка': 5897,
         ' ': 33570,
         'жилье': 4460,
         '\n': 21454,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6351,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 478,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'подержанный': 486,
         'получение': 1314,
         'высокий': 1374,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607})

In [10]:
display(data['purpose'].value_counts())

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [11]:
def purpose_group (lemmas):
    if 'жиль' in lemmas or 'недвижим' in lemmas:
        return 'недвижимость'
    if 'авто' in lemmas:
        return 'автомобили'
    if 'образов' in lemmas:
        return 'обучение'
    if 'свадь' in lemmas:
        return 'свадьба'
    return 'потерял данные'
   
 
data['purpose_id'] = data['purpose'].apply(purpose_group)

display(data['purpose_id'].value_counts())


недвижимость    10811
автомобили       4306
обучение         4013
свадьба          2324
Name: purpose_id, dtype: int64

**Вывод** С помощью лемм выделили цели и кол-во их

### Категоризация данных

In [12]:
def children_group(children):
    if children == 0:
        return 'бездетные'
    if children <= 2:
        return 'с детьми'
    return 'многодетные'
data['children_category'] = data['children'].apply(children_group)
data['children_category'].value_counts()

бездетные      14091
с детьми        6907
многодетные      456
Name: children_category, dtype: int64

In [13]:
def income_group(row):
    total_income = row['total_income']
    if 0 <= total_income <= 100000:
        return 'низкий уровень дохода'
    if 100001 <= total_income <= 250000:
        return 'средний уровень дохода'
    return 'высокий уровень дохода'

data['income_category'] = data.apply(income_group, axis=1)
display(data['income_category'].value_counts())

средний уровень дохода    14178
низкий уровень дохода      4463
высокий уровень дохода     2813
Name: income_category, dtype: int64

In [14]:
def status_group(family_status_id):
    if family_status_id == 0:
        return 'в браке'
    if family_status_id == 1:
        return 'гражданский брак'
    if family_status_id == 2:
        return 'вдовец/вдова'
    if family_status_id == 3:
        return 'в разводе'
    return 'не жента / не замужем'
data['status_category'] = data['family_status_id'].apply(status_group)
data['status_category'].value_counts()

в браке                  12339
гражданский брак          4151
не жента / не замужем     2810
в разводе                 1195
вдовец/вдова               959
Name: status_category, dtype: int64

**Вывод** С помощью категоризации данных выделили важные значения для последующего ответа на заданные вопросы

## Шаг 3. Ответы на вопросы:
    Есть ли зависимость между наличием детей и возвратом кредита в срок?
    Есть ли зависимость между семейным положением и возвратом кредита в срок?
    Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
    Как разные цели кредита влияют на его возврат в срок?
    

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [15]:
data_pivot_ch = data.pivot_table(index=['children_category'], columns = 'debt', values = 'children', aggfunc = 'count')
data_pivot_ch



debt,0,1
children_category,,
бездетные,13028,1063
многодетные,417,39
с детьми,6268,639


In [16]:
children_conversion = data_pivot_ch[1] / (data_pivot_ch[0] + data_pivot_ch[1]) * 100
children_conversion

children_category
бездетные      7.543822
многодетные    8.552632
с детьми       9.251484
dtype: float64

**Вывод** Категория с детьми и многотодетные вовзвращают кредит невовремя по сравнению с категорией без детей


### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [17]:
data_pivot_st = data.pivot_table(index=['status_category'], columns = 'debt', values = 'family_status_id', aggfunc = 'count')
data_pivot_st

debt,0,1
status_category,,
в браке,11408,931
в разводе,1110,85
вдовец/вдова,896,63
гражданский брак,3763,388
не жента / не замужем,2536,274


In [18]:
status_conversion = data_pivot_st[1] / (data_pivot_st[0] + data_pivot_st[1]) * 100
status_conversion.sort_values()

status_category
вдовец/вдова             6.569343
в разводе                7.112971
в браке                  7.545182
гражданский брак         9.347145
не жента / не замужем    9.750890
dtype: float64

**Вывод** Не женатые/не замужние и в гражданском браке имеют плохую статистику по возврату кредита в срок

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [19]:
data_pivot_ti = data.pivot_table(index=['income_category'], columns = 'debt', values = 'total_income', aggfunc = 'count')
data_pivot_ti

debt,0,1
income_category,,
высокий уровень дохода,2619,194
низкий уровень дохода,4109,354
средний уровень дохода,12985,1193


In [20]:
income_conversion = data_pivot_ti [1] / (data_pivot_ti [0] + data_pivot_ti [1]) * 100
income_conversion

income_category
высокий уровень дохода    6.896552
низкий уровень дохода     7.931884
средний уровень дохода    8.414445
dtype: float64

**Вывод** Поразительно, люди с низким уровнем дохода обошли группу лиц со средним уровнем по возврату кредита в срок

### Как разные цели кредита влияют на его возврат в срок?

In [21]:
data_pivot_pr = data.pivot_table(index=['purpose_id'], columns = 'debt', values = 'purpose', aggfunc = 'count')
data_pivot_pr

debt,0,1
purpose_id,,
автомобили,3903,403
недвижимость,10029,782
обучение,3643,370
свадьба,2138,186


In [22]:
purpose_conversion = data_pivot_pr[1] / (data_pivot_pr[0] + data_pivot_pr[1]) * 100
purpose_conversion.sort_values()

purpose_id
недвижимость    7.233373
свадьба         8.003442
обучение        9.220035
автомобили      9.359034
dtype: float64

**Вывод** 

Автомобиль и обучение опаздывают с возвратом кредита в срок, тоже неожиданно, думал в лидерах будет недвижимость...

## Шаг 4. Общий вывод

**Вывод** 
1. Зависимости, которые удалось вычислить, для меня были неожиданны, процент не возврата кредита в срок намного больше, а так же группа лиц с низким уровнем дохода намного лучше осуществляют возврат кредита в срок, чем другие группы.
2. Пропущенные значения в 1ом моем варианте я заменил на 0, что оказалось неверно, так как терялись важные значения для последующего подсчета категорий и их зависимостей, во 2ом варианте произвел замену пропущенных значений на средние значения, данные изменились.
3. С помощью категоризации данных мы разделили нужный нам столбец на категории и вычислили кол-во нужны нам значений для последующего ответа на посталвенные вопросы, как та или ина категория групп людей связана между собой.